# CityShadows: A Shade-based Route Recommendation Tool (Dynamic)

This notebook loads building footprints with heights, and tree data from an area in Makati, Metro Manila. It takes a pair of start-end coordinates and recommends a route with the best amount of shade along it given a date and time. Unlike `city_shadows.ipynb`, it accounts for longer trips, where the travel time is sufficient for shadows to shift along the route.

### Requirements:
- **Python 3.9** in order for pybdshadow to function properly.
- .env file containing an **Openrouteservice** (https://openrouteservice.org/) API key (OSM_API)

## Libraries

In [ ]:
from dotenv import load_dotenv
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import numpy as np
import openrouteservice
import pandas as pd
import polyline
import pybdshadow
from pyproj import Geod
import random
from shapely.geometry import Point, LineString, box
from shapely import LineString
import urllib, os
import urllib.request

## Variables

### shp files

In [ ]:
small_makati = gpd.read_file("../data/smaller_makati.shp")
buildings = gpd.read_file("../data/gis_osm_buildings_a_free_1.shp")
less_makati_buildings = buildings[small_makati['geometry'].item().contains(buildings['geometry'])]
roads = gpd.read_file("../data/gis_osm_roads_free_1.shp")
less_makati_roads = roads[small_makati['geometry'].item().contains(roads['geometry'])]

### geojson files (buildings + trees)

In [ ]:
buildings_dataset = gpd.read_file("mapping_heights/buildings_dataset.geojson")
trees_dataset = gpd.read_file("mapping_heights/trees_dataset.geojson")

In [ ]:
buildings_gdf4326 = buildings_dataset.to_crs("EPSG:4326")
trees_gdf4326     = trees_dataset.to_crs("EPSG:4326")

### api

In [ ]:
load_dotenv(override=True)
OSM_API = os.getenv('OSM_API')

### start & end coordinates

In [ ]:
startLat, startLong = 121.010714, 14.556621
endLat, endLong = 121.024467, 14.552471

### date & time
Time interval refers to the amount of minutes walking before the shadows update.

In [ ]:
date_input = '2025-03-07 09:00:00'
timeInterval=30

### Walking speed in meters (relative to given time interval)

In [ ]:
distancePerTimeInterval=1000

## Function Declaration

### Helpers

In [ ]:
def convert_ph_to_utc(ph_datetime):
    """
    Converts a datetime in Philippine time to UTC.
    
    Parameters:
      ph_datetime (str or datetime-like): A datetime in Philippine time.
    
    Returns:
      A pandas.Timestamp representing the time in UTC.
    """
    ts = pd.to_datetime(ph_datetime)
    ts_utc = ts.tz_localize('Asia/Manila').tz_convert('UTC')
    return ts_utc

### Routing

In [ ]:
def getOSMRoutes(startLat, startLong, endLat, endLong, apiKey):
    """
    Retrieves up to 6 alternative walking routes between two coordinates using the OpenRouteService API.

    The function makes two routing requests:
    1. With default ORS alternative route settings (weight_factor=1.4, share_factor=0.6).
    2. With custom settings (weight_factor=2, share_factor=0.2, preference='shortest').

    It deduplicates the results based on the route geometry to ensure only unique routes are returned.

    Parameters:
        startLat (float): Latitude of the start location.
        startLong (float): Longitude of the start location.
        endLat (float): Latitude of the end location.
        endLong (float): Longitude of the end location.
        apiKey (str): OpenRouteService API key.

    Returns:
        list of dict: A list of route dictionaries, each containing:
            - 'coordinates': list of (lat, lon) tuples representing the route path.
    """
    client = openrouteservice.Client(key=apiKey)

    # get first 3 routes with default ORS settings (weight factor=1.4 and share factor=0.6)
    requestParams1 = {'coordinates': [[startLat, startLong],
                                       [endLat, endLong]],
                      'profile': 'foot-walking',
                      'alternative_routes': {
                          'target_count': 3,
                          'weight_factor': 1.4,
                          'share_factor': 0.6
                      }}
    # get next 3 routes with custom ORS settings 
    requestParams2 = {'coordinates': [[startLat, startLong],
                                       [endLat, endLong]],
                      'profile': 'foot-walking',
                      'alternative_routes': {
                          'target_count': 3,
                          'weight_factor': 2,
                          'share_factor': 0.2
                      }, 
                      'preference': 'shortest'
                      }

    routeRequest1  = client.directions(**requestParams1)
    routeRequest2  = client.directions(**requestParams2)

    # set to store unique geometry strings
    unique_geometries = set()
    routeData = []
    routeDistance=[]

    for routeRequest in [routeRequest1, routeRequest2]:
        for route in routeRequest.get("routes", []):
            geometry_str = route["geometry"]  
            if geometry_str not in unique_geometries: 
                unique_geometries.add(geometry_str)
                routeData.append(polyline.decode(geometry_str))
                routeDistance.append(route['summary']['distance'])
    return routeData,routeDistance


def convert_routes_to_linestrings(routes):
    """
    Converts a list of route dictionaries into Shapely LineString objects.

    This function takes decoded route coordinates (in (lat, lon) format)
    and transforms them into LineString geometries (in (lon, lat) format)
    for spatial analysis and GIS compatibility.

    Parameters:
        routes (list of dict): A list of route dictionaries with a key "coordinates"
            containing a list of (lat, lon) tuples.

    Returns:
        list of shapely.geometry.LineString: A list of LineString objects representing the routes.
    """

    linestrings = []
    for route in routes:
        coords = LineString([(lon, lat) for (lat, lon) in route])
        linestrings.append(coords)
    return linestrings

In [ ]:
geod = Geod(ellps="WGS84")

def distance_geodesic(p1, p2):
    _, _, dist = geod.inv(p1[0], p1[1], p2[0], p2[1])
    return dist

def interpolate_geodesic(p1, p2, distance):
    az12, _, _ = geod.inv(p1[0], p1[1], p2[0], p2[1])
    lon, lat, _ = geod.fwd(p1[0], p1[1], az12, distance)
    return (lon, lat)

def extract_first_n_meters(line, limit_m):
    coords = list(line.coords)
    if len(coords) < 2:
        return LineString(coords)  # not enough data to walk

    new_coords = [coords[0]]
    total = 0.0

    for i in range(1, len(coords)):
        p1 = coords[i - 1]
        p2 = coords[i]
        seg_len = distance_geodesic(p1, p2)

        if total + seg_len >= limit_m:
            remaining = limit_m - total
            cutoff_point = interpolate_geodesic(p1, p2, remaining)
            new_coords.append(cutoff_point)
            break
        else:
            new_coords.append(p2)
            total += seg_len

    return LineString(new_coords)

### Detecting nearby buildings

In [ ]:
def extractStructuresNearRoad(road, structureDF, threshold):
    """
    Returns structures within the specified distance threshold from any road LineString.
    
    Parameters:
        road (list of LineStrings): List of road geometries.
        structureDF (GeoDataFrame): GeoDataFrame of structures (must have geometry column).
        threshold (float): Distance threshold in degrees (e.g., 0.0005 ≈ 55 meters).
    
    Returns:
        GeoDataFrame: Structures near any of the road lines.
    """
    # Use apply to check distance from each building to any of the LineStrings
    nearby_structures = structureDF[structureDF.geometry.apply(
        lambda geom: any(r.distance(geom) <= threshold for r in road)
    )]
    return nearby_structures


def plotBuildingNearRoad(road, buildingDF, adjacentBuildingsDF):
    """
    Plots all buildings, highlights adjacent buildings in red, and road routes in blue.
    
    Parameters:
        road (list of LineStrings): List of route geometries.
        buildingDF (GeoDataFrame): All buildings.
        adjacentBuildingsDF (GeoDataFrame): Filtered buildings near road.
    """
    # Convert input to GeoDataFrame if needed
    adjacent_buildings = gpd.GeoDataFrame(adjacentBuildingsDF, crs="EPSG:4326")
    adjacent_buildings.set_geometry('geometry', inplace=True)
    
    # Combine road geometries into a GeoSeries for plotting
    sample_road = gpd.GeoSeries(road, crs="EPSG:4326")

    # Plot
    ax = buildingDF.plot(color='lightgray', edgecolor='black', figsize=(10, 10))
    adjacent_buildings.plot(ax=ax, color='red', label='Adjacent Buildings')
    sample_road.plot(ax=ax, color='blue', label='Road')
    
    plt.legend()
    plt.title("Buildings Near Road")
    plt.show()


### Shadow calculation

In [ ]:
def tryGeneratingShadow(structures, date):
    # Define a flag that forces the area to be marked as shaded if the time is outside daylight.
    force_shade = True
    is_daylight = True      # to determine if it is before sunrise or after sunset

    # Try to calculate shadows. If the time is before sunrise or after sunset, handle based on force_shade.
    try:
        shadows = pybdshadow.bdshadow_sunlight(
            structures, 
            date, 
            height='height',      
            roof=False, 
            include_building=True, 
            ground=0
        )
    except ValueError as e:
        if "before sunrise or after sunset" in str(e) and force_shade:
            print("Time is outside daylight hours. Marking entire area as shaded (force_shade=True).")
            shadows = structures.copy()  # Entire area is treated as shaded.
            is_daylight = False
        else:
            raise
    return shadows, is_daylight


def compute_experienced_distance(route, shadows_gdf, sun_averion):
    """
    Computes the experienced distance of a route.

    Parameters:
      route (shapely.geometry.LineString): The route geometry in EPSG:4326.
      shadows_gdf (GeoDataFrame): A GeoDataFrame of shadow geometries in EPSG:4326.
      sun_aversion (int): a value >=1 that increases the experienced distance if more unshaded paths are walked on.

    Returns:
      float: Total experienced distance.
    """
    # Project the route to a metric CRS for length accuracy
    route_proj = (
        gpd.GeoSeries([route], crs="EPSG:4326")
           .to_crs(epsg=3857)
           .iloc[0]
    )

    # Ensure shadows_gdf has a CRS, then project to the same metric CRS
    if shadows_gdf.crs is None:
        shadows_gdf = shadows_gdf.set_crs(epsg=4326)
    shadows_proj = shadows_gdf.to_crs(epsg=3857)

    # Union all shadows into one geometry
    union_shadow = shadows_proj.geometry.union_all()

    # Compute intersection of the route with the shadow union
    intersection = route_proj.intersection(union_shadow)

    # Measure lengths
    shaded_length = intersection.length if not intersection.is_empty else 0.0
    total_length  = route_proj.length
    experienced_length = (total_length-shaded_length)*sun_averion

    # Return shaded percentage
    return shaded_length+experienced_length if total_length > 0 else 0.0


## Dynamic Shadow Calculation
- gets the available routes
- trims the routes to the distance traveled after `timeInterval` minutes specified in the variables
- chooses the best route
- repeats the process starting from that route until the end point is reached
<br><br>
for more information on how each part of the code snippet works, refer to the `FINAL.ipynb` 

In [ ]:
tempStartLat, tempStartLong = startLat,startLong
date = pd.to_datetime(convert_ph_to_utc(date_input))

bestRouteSegment =[]
collectedRoutes=[]
collectedBestIdx=[]
collectedBuildings=[]
collectedTrees=[]
collectedBuildingShadows=[]
collectedTreeShadows=[]
collectedBestDistance=[]
totalExperiencedDistance=0

while tempStartLat != endLat and tempStartLat != endLong:
    print(f"curr start lat long: {tempStartLat}, {tempStartLong} with endpoint {endLat}, {endLong}")
    # get first n meters of the route
    routes,distances = getOSMRoutes(tempStartLat, tempStartLong, endLat, endLong, OSM_API)
    routes = convert_routes_to_linestrings(routes)
    routePerHour = []
    for r in routes:
        routePerHour.append(extract_first_n_meters(r,distancePerTimeInterval))

    # extract buildings near the routes
    adjacent_buildings = extractStructuresNearRoad(road = routePerHour, 
                                                structureDF = buildings_gdf4326, 
                                                threshold = 0.0005)
    adjacent_trees = extractStructuresNearRoad( road = routePerHour,
                                                structureDF = trees_gdf4326,
                                                threshold   = 0.0005) 

    # process buildings for shadow calculation                     
    processed_buildings = pybdshadow.bd_preprocess(adjacent_buildings, height='height')
    processed_trees = pybdshadow.bd_preprocess(adjacent_trees, height='height')

    # store vals
    collectedBuildings.append(processed_buildings)
    collectedTrees.append(processed_trees)

    shadows_buildings, is_daylight = tryGeneratingShadow(processed_buildings, date)
    shadows_trees, is_daylight = tryGeneratingShadow(processed_trees, date)

    # store vals
    collectedBuildingShadows.append(shadows_buildings)
    collectedTreeShadows.append(shadows_trees)

    routes_gdf= gpd.GeoDataFrame(geometry=routePerHour, crs="EPSG:4326")
    collectedRoutes.append(routes_gdf)

    # Build the combined shaded areas (buildings + tree shadows + tree canopies)
    shaded_areas = gpd.GeoDataFrame(
        pd.concat([
            shadows_buildings,
            shadows_trees,
            processed_trees[['geometry']].rename(columns={'geometry':'geometry'})
        ], ignore_index=True),
        crs=shadows_buildings.crs
    )

    # Compute and print shaded percentage for each route
    exp_distances = []
    print("Experienced distance per route:")
    for idx, route in enumerate(routes_gdf.geometry):
        exp_distance = compute_experienced_distance(route, shaded_areas, 5)
        exp_distances.append(exp_distance)
        print(f"  Route {idx}: {exp_distance:.2f}")

    if exp_distances[0] < 1:
        break

    # Tie‐breaker logic to pick the best route
    best_idx = int(np.argmin(exp_distances))
    collectedBestDistance.append(exp_distances[best_idx])
    totalExperiencedDistance+=exp_distances[best_idx]
    collectedBestIdx.append(best_idx)
    lengths = routes_gdf.geometry.length
    dupes = [(i, p, lengths[i]) for i, p in enumerate(exp_distances) if p == exp_distances[best_idx]]
    if len(dupes) > 1:
        # choose shortest among the tied routes
        best_idx = dupes[np.argmin([d[2] for d in dupes])][0]

    # init for next run
    bestRouteSegment.append(routes_gdf.iloc[[best_idx]])

    # visualizing the iter
    fig, ax = plt.subplots(figsize=(10, 10))

    # all routes light gray
    for i,r in enumerate(collectedRoutes):
        if i==0:
            r.plot(ax=ax, color='orange', linewidth=1, label='Other Routes')
        else:
            r.plot(ax=ax, color='orange', linewidth=1)

    # highlight best route
    for i,br in enumerate(bestRouteSegment):
        if i==0:
            collectedRoutes[i].iloc[[collectedBestIdx[i]]].plot(
                ax=ax, color='red', linewidth=3,
                label=f'Chosen Route ({totalExperiencedDistance:.1f} distance experienced.)'
            )
        else:
            collectedRoutes[i].iloc[[collectedBestIdx[i]]].plot(
                ax=ax, color='red', linewidth=3)

    # buildings + their shadows
    for b in collectedBuildings:
        b.plot(ax=ax, color='blue', alpha=0.7, label='Buildings')
    for bs in collectedBuildingShadows:
        bs.plot(ax=ax, color='gray', alpha=0.5, label='Building Shadows')

    # trees + their shadows
    for t in collectedTrees:
        t.plot(ax=ax, color='darkgreen', alpha=0.7, label='Trees')
    for ts in collectedTreeShadows:
        ts.plot(ax=ax, color='lightgreen', alpha=0.2, label='Tree Shadows')

    ax.legend()
    ax.set_title(f"Best Route with Both Building & Tree Shading for {date}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.tight_layout()
    plt.show()

    lastIdx = len(routePerHour[best_idx].coords.xy[0])-1
    tempStartLat, tempStartLong = routePerHour[best_idx].coords.xy[0][lastIdx], routePerHour[best_idx].coords.xy[1][lastIdx]
    date += pd.Timedelta(minutes=timeInterval)